# Intro
When we deal with photos, we usually hope to get all the necessary information from the image itself using CNN or other features, but in this contest I don't think this is the best way to get the maximum score, case, as many of us have noticet, that the photos do not represent a real Pawpularity score. So I'm trying to find proxy characteristics with high correlation with the target.
In this notebook, I extract the features that we lose when preparing photos for CNN, such as image size, height and weight.

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import image
import os
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts

In [ ]:
train_path = '/kaggle/input/petfinder-pawpularity-score/train/'

In [ ]:
data = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")

In [ ]:
d = ["_size_1", '_size_2', "st_size", "format", 'size']
d = {i:[] for i in d}
y = 0
for i in data["Id"]:
    img_path = os.path.join(train_path, i+".jpg")
    x = os.stat(img_path)
    file_pr = {k: getattr(x, k) for k in dir(x) if k in d}
    for i in file_pr:
        d[i].append(file_pr[i])
    
    img = image.load_img(img_path)
    img_pr = img.__dict__
    try:
        _size_1, _size_2 = img_pr.get("_size")
    except:
        print(img_pr.get("_size"))
    d["_size_1"].append(_size_1)
    d['_size_2'].append(_size_2)
    d['size'].append(str(_size_1)+str(_size_2))
    
    if _size_1==_size_2:
        fm =  "sqr"
    elif _size_1<_size_2:
        fm =  "horiz"
    elif _size_1>_size_2:
        fm =  "vert"
    d["format"].append(fm)    

In [ ]:
img_info = pd.DataFrame.from_dict(d)

In [ ]:
train = pd.concat([data, img_info], axis=1)

In [ ]:
y = train['Pawpularity']
X = train.drop(["Pawpularity", "Id"],axis=1)
X_train, X_test, Y_train, Y_test = tts(X, y, test_size=0.2, random_state=2)

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=100000,
#                           learning_rate=0.001,
                          depth=3)
# cat_features_ids = ["Id"]
model.fit(X_train, Y_train,verbose=1, eval_set=[(X_test, Y_test)], early_stopping_rounds=1000, metric_period=1000, cat_features=['size', 'format'])
y_pred = model.predict(X_test)
# y_pred = y_pred.round()

In [ ]:
plt.figure(figsize= (15, 15))
sns.heatmap(train.corr(), annot=True, fmt='.1g' )
plt.title('Correlation Matrix', fontweight='bold', fontsize=20)
plt.show()

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, features=X, feature_names=X.columns)

# Conclusion
As we can see on the shap plot, the additional characteristics have a great impact on the model evaluation, and st_size, _size_1 and size exceed all the original metadata